# Generacion de la red

In [1]:
#Paquetes que importamos
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Cargaremos en este caso un modelo de red neuronal ya entrenado (ResNet50V2)
model = keras.applications.ResNet50V2(weights="imagenet", include_top=False) #Tomamos los pesos de la red entrenada con imagenet y sin incluir la capa fullyconnected


In [33]:
model.summary()

In [31]:
#Capa de la que veremos la salida
capa = "conv5_block3_out"#"conv2_block2_out" #"conv5_block3_out" #"conv3_block4_out"

#Tomamos la capa del modelo y definimos su salida como el extractor de caracteristicas
Capa_salida = model.get_layer(name=capa)
Extractor_caracteristicas = keras.Model(inputs=model.inputs, outputs=Capa_salida.output)

# Funciones para visualizar el proceso de entrenamiento

In [4]:
def calculo_perdida(imagen_de_entrada, indice_filtro): #Computa el error que se obtiene sobre la imagen
    
    #Calculamos la salida que genera la imagen de entrada
    activacion = Extractor_caracteristicas(imagen_de_entrada)
    
    #Y tomamos los resultados del filtro de convolucion que vamos a visualizar
    filtro = activacion[:, 2:-2, 2:-2, indice_filtro]
    
    #Y devolvemos el error (su media)
    return tf.reduce_mean(filtro)

def paso_gradiente(imagen, indice_filtro, tasa_aprendizaje):
    
    #La Funcion gradientTape() computa el gradiente sobre la imagen
    with tf.GradientTape() as gradiente:
        #Dejamos que el gradiente vea la imagen y calcule la perdida
        gradiente.watch(imagen)
        perdida = calculo_perdida(imagen, indice_filtro)
    
    #Calculamos el gradiente.
    grad = gradiente.gradient(perdida, imagen)
    
    #Y lo normalizamos
    grad = tf.math.l2_normalize(grad)
    
    #Despues actualizamos lo aprendido en la imagen
    imagen += tasa_aprendizaje * grad
    return perdida, imagen

In [5]:
def inicializa_imagen():
    # Empezamos con una imagen de 180x180 gris con ruido; con los valores de -1 a 1 y la semilla 3
    imagen = tf.random.uniform((1, 180, 180, 3))

    return imagen 

def visualizacion_filtro(indice_filtro):
    
    # Ejecutamos 30 iteraciones del descenso de gradiente
    iteraciones = 30
    tasa_aprendizaje = 10.0
    imagen = inicializa_imagen()
    for iteracion in range(iteraciones):
        perdida, imagen = paso_gradiente(imagen, indice_filtro, tasa_aprendizaje)

    #Decodificamos los resultados
    imagen = decodificacion(imagen[0].numpy())
    return perdida, imagen

def decodificacion(imagen):
    
    # Normalizamos la imagen
    imagen -= imagen.mean()
    imagen /= imagen.std() + 1e-5
    imagen *= 0.15

    # La centramos
    imagen = imagen[25:-25, 25:-25, :]
    imagen += 0.5
    imagen = np.clip(imagen, 0, 1)

    # Y la convertimos a RGB
    imagen *= 255
    imagen = np.clip(imagen, 0, 255).astype("uint8")
    return imagen

## Visualizacion resultados

In [42]:
#Paquete para graficar datos
import matplotlib.pyplot as plt

#Métodos para pintar
#Pintamos los píxeles
plt.matshow(tf.random.uniform((1, 180, 180, 3))[0].numpy(), cmap = plt.cm.gray)
#Eliminamos los ejes
plt.xticks(())
plt.yticks(())
#Mostramos la gráfica
plt.show

In [35]:
from IPython.display import Image, display

perdida, imagen = visualizacion_filtro(0)
keras.preprocessing.image.save_img("2conv.png", imagen)
display(Image("2conv.png"))